In [2]:
1+1

2

In [3]:
import geopandas as gpd
gdf = gpd.read_file('../data/inputs/h3.gpkg', layer='h3')
gdf = gdf.to_crs("EPSG:4326")
gdf.shape

(2474848, 6)

Export the layer as WGS84 GeoJSON from the running notebook (ensure lon/lat)

In [11]:
gdf.head()

,GRID_ID,within0km,within10km,MERGE_SRC,level,geometry
0,859c20a7fffffff,NaN,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,5,"MULTIPOLYGON (((145.99136 -18.04283, 145.97602..."
1,859c20b7fffffff,1.0,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,5,"MULTIPOLYGON (((146.09738 -18.1589, 146.08191 ..."
2,859c2143fffffff,1.0,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,5,"MULTIPOLYGON (((145.57041 -17.57988, 145.55557..."
3,859c2147fffffff,NaN,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,5,"MULTIPOLYGON (((145.40576 -17.60918, 145.39108..."
4,859c214bfffffff,1.0,NaN,Y:\Projects\Ben\ForestHealth\ForestHealth.gdb\...,5,"MULTIPOLYGON (((145.63037 -17.43513, 145.61549..."


In [4]:
gdf.describe()

,within0km,within10km,level
count,2.421087e+06,2413646.0,2.474848e+06
mean,8.372520e-02,1.0,9.945876e+00
std,2.769753e-01,0.0,3.479752e-01
min,0.000000e+00,1.0,5.000000e+00
25%,0.000000e+00,1.0,1.000000e+01
50%,0.000000e+00,1.0,1.000000e+01
75%,0.000000e+00,1.0,1.000000e+01
max,1.000000e+00,1.0,1.000000e+01


In [12]:
gdf.drop(columns=['within10km', 'MERGE_SRC'], inplace=True)

In [14]:
gdf.to_file("h3.geojson", driver="GeoJSON")

Build MBTiles with tippecanoe:

```bash
# choose min/max zoom that fit your H3 levels; adjust -Z / -z
tippecanoe -o h3.mbtiles -l h3 \
  -Z5 -z12 \
  --generate-ids \
  --drop-densest-as-needed \
  h3.geojson
```

```bash
# choose min/max zoom that fit your H3 levels; adjust -Z / -z
tippecanoe -o h3.mbtiles -l h3 \
  -zg \
  --generate-ids \
  --drop-densest-as-needed \
  h3.geojson
```

```bash
tippecanoe -o h3.pmtiles -l h3 \
  -Z5 -z12 \
  --generate-ids \
  --order-by level \
  --no-tile-size-limit \
  h3.geojson
```

Key flags:

- `--order-by area` — sorts features so larger hexagons render first (underneath), smaller ones on top. Adjust if you want reverse: --order-by area --reverse-order
- `--no-tile-size-limit` — prevents tippecanoe from dropping features to keep tile size down (you need all polygons for DB filtering)

```bash
tippecanoe -o h3.mbtiles -l h3 \
  -Z5 -z12 \
  --generate-ids \
  --order-by level \
  --no-tile-size-limit \
  h3.geojson
```

Notes / tips

- Keep geometry properties you need (e.g. GRID_ID) so tiles contain identifiers you can use to filter your DB. tippecanoe preserves feature properties.
- If you want deterministic feature IDs in tiles, include id property or use --generate-ids.
- Tune -Z/-z, --drop-densest-as-needed and --simplification flags to control tile size/detail.
- Client-side libraries to compute tiles from a map view: mercantile (Python), @mapbox/tile-cover or tilebelt (JS).

----

Serve the MBTiles (examples):

```bash
# simple local serving with tileserver-gl (Docker)
docker run --rm -v ${PWD}:/data -p 8080:80 maptiler/tileserver-gl h3.mbtiles
# or use tileserver-gl-light / any mbtiles server
```

In the client map: compute the set of vector tiles covering the current map bbox at the current zoom and request those / use them to filter your DB. Example (Python server side) using mercantile to get z/x/y tiles for a bbox:

```python
import mercantile
# bbox = (west, south, east, north) from map view
tiles = list(mercantile.tiles(west, south, east, north, [zoom]))
# tiles is an iterator of Tile(x,y,z); use this list to query DB (e.g. filter by tile id or intersect)
```

----

Convert from MB tiles to PM tiles

`npm install -g pmtiles`

`pmtiles convert input.mbtiles output.pmtile`

OR just export *.pmtiles from tippecanoe

----

To serve the tiles from localhost do the following: `cp ../code/my_code/satellite-data-toolbox/h3.pmtiles .` & `npx http-server . --cors -p 8080`